In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np

In [ ]:
pip install matplotlib 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
dp = Path('./src/tables/recipes_model/')


1. Загрузите данные из файлов `reviews_short.csv` и `recipes_short_5.csv` в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id, name, minutes, submitted, description, n_ingredients`


In [ ]:
from google.colab import files 
uploaded = files.upload()

Saving recipes_sample.csv to recipes_sample.csv


In [ ]:
from google.colab import files 
uploaded = files.upload()

Saving reviews_sample (2).csv to reviews_sample (2).csv


In [ ]:
reviews = pd.read_csv('reviews_sample(2).csv', index_col=0)
recipes = pd.read_csv('recipes_sample.csv')
recipes = recipes.drop(columns=['n_steps', 'contributor_id'])
recipes


FileNotFoundError: ignored

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл recipes.xlsx. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц.


In [ ]:
reviews_s = reviews.sample(len(reviews)//20)
reviews_s


NameError: ignored

In [ ]:
recipes_s = recipes.sample(len(reviews)//20)
recipes_s


NameError: ignored

In [ ]:
writer = pd.ExcelWriter("./src/tables/recipes.xlsx", engine='xlsxwriter')
recipes_s.to_excel(writer, sheet_name='Рецепты')
reviews_s.to_excel(writer, sheet_name='Отзывы')
writer.save()


ModuleNotFoundError: ignored

In [ ]:
import xlwings as xw
wb = xw.Book("./src/tables/recipes.xlsx")
sheet = wb.sheets['Рецепты']


ModuleNotFoundError: ignored

3. Используя xlwings, добавьте на лист Рецепты столбец seconds_assign, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.


In [ ]:
sheet['H1'].value = ['seconds_assign', *
                     (recipes_s['minutes']*60).values.reshape((-1, 1))]


NameError: ignored

4. Используя xlwings, добавьте на лист Рецепты столбец seconds_formula, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.


In [ ]:
sheet[f'I1'].value = [['seconds_formula']] + \
    [[f'=D{n+2}*60'] for n in range(len(recipes_s))]


NameError: ignored

5. Добавьте на лист Рецепты столбец n_reviews, содержащий кол-во отзывов для этого рецепта


In [ ]:
ids = list(reviews_s['recipe_id'].values)
sheet['J1'].value = [['n_reviews'], *[[ids.count(i)] for i in ids]]

NameError: ignored

6. Раскрасьте ячейки столбца minutes в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.


In [ ]:
def recolor(sheet, cells, color, /, step=40):
    index = 0
    while True:
        n_index = None if (index + step > len(cells)) else index + step
        sheet[';'.join(cells[index:n_index])].color = color
        # print(index, n_index)
        index += step
        if not n_index:
            break


In [ ]:
mins = recipes_s['minutes'].values

green_rows = [f'D{i}' for i in np.where(mins < 5)[0]+2]
recolor(sheet, green_rows, "#00ff00")

red_rows = [f'D{i}' for i in np.where(mins > 10)[0]+2]
recolor(sheet, red_rows, "#ff0000")

yellow_rows = [f'D{i}' for i in np.where((5 <= mins) & (mins <= 10))[0]+2]
recolor(sheet, yellow_rows, "#ffff00")

NameError: ignored

7. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.


In [ ]:
titles = 'A1:J1'
sheet[titles].api.Font.FontStyle = 'Bold'
sheet[titles].api.VerticalAlignment = 2
sheet[titles].api.HorizontalAlignment = 3
sheet['A:J'].autofit()

8. Напишите функцию validate(), которая проверяет соответствие всех строк из листа interactions следующим правилам:\
   Рейтинг - это число от 0 до 5 включительно\
   Соответствующий рецепт имеется на листе рецептов\
   В случае несоответствия этим правилам, выделите строку красным цветом


In [ ]:
def validate(sheet):
    rating = reviews_s['rating'].values
    ids_rec = recipes_s['id'].values
    ids_rev = reviews_s['recipe_id'].values
    [f'J{n}' for n, i in enumerate(rating) if not (0 <= i <= 5)]

    if recol := [f'J{n}' for n, i in enumerate(rating) if not (0 <= i <= 5) or (ids_rev[n] not in ids_rec)]:
        recolor(sheet, recol, '#ff0000')

validate(sheet)